1\. **2D minimization of a six-hump camelback function**

$$f(x,y) = \left(4-2.1x^2+\frac{x^4}{3} \right) x^2 +xy + (4y^2 -4)y^2$$

has multiple global and local minima.

- Find the global minima of this function
- How many global minima are there, and what is the function value at those points?
- What happens for an initial guess of $(x, y) = (0, 0)$?

Hints:

* Variables can be restricted to $-2 < x < 2$ and $-1 < y < 1$.
* Use `numpy.meshgrid()` and `pylab.imshow()` to graphically display the regions.
* Use `scipy.optimize.minimize()`, trying its optional arguments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def f(x, y):
    return (4 - 2.1 * x**2 + x**4 / 3) * x**2 + x * y + (4 * y**2 - 4) * y**2

x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 1, 200)
X, Y = np.meshgrid(x, y)

Z = f(X, Y)

plt.imshow(Z, extent=[-2, 2, -1, 1], origin='lower', cmap='viridis')
plt.colorbar()
plt.title("Regions of Global Minima")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

initial_guess = [(0, 0), (1, 1), (-1, -1)]  # Try different initial guesses

# Define the objective function to minimize
objective = lambda xy: f(xy[0], xy[1])

for guess in initial_guess:
    result = minimize(objective, guess, bounds=((-2, 2), (-1, 1)))
    print(f"Global Minimum at ({result.x[0]:.3f}, {result.x[1]:.3f}): {result.fun:.3f}")

2\. **Non-linear ODE: the damped pendulum**

The equation of the motion of a forced pendulum, as a function of the angle $\theta$ with the vertical, is given by:

$$\frac{d^2\theta}{dt^2} = -\frac{1}{Q} \frac{d\theta}{dt} + \frac{g}{l}\sin\theta + d \cos\Omega t$$

where $t$ is time, $Q$ is the damping factor, $d$ is the forcing amplitude, and $\Omega$ is the driving frequency of the forcing. 

This second order ODE needs to be written as two coupled first order ODEs by defining a new variable $\omega \equiv d\theta/dt$:

$$\frac{d\theta}{dt} = \omega$$
$$\frac{d\omega}{dt} = -\frac{1}{Q}\,\omega + \frac{g}{l}\sin\theta + d \cos\Omega t$$

Consider the initial conditions $\theta_0 = \omega_0 = 0$, and $l = 10$, $Q = 2.0$, $d = 1.5$, and $\omega = 0.65$.

 - Solve the ODE with `odeint` over a period of 200 time steps
 - Create two plots, one of $\theta$ as a function of the time, and $\omega$ as a function of the time
 - **Optional**: determine if there is a set of parameters for which the motion is chaotic.

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Define the function that represents the system of first-order ODEs:
def pendulum(y, t, Q, d, g, l, Omega):
    theta, omega = y
    d_theta_dt = omega
    d_omega_dt = -(1/Q) * omega + (g/l) * np.sin(theta) + d * np.cos(Omega * t)
    return [d_theta_dt, d_omega_dt]

# Set the initial conditions and other parameters
theta_0 = 0
omega_0 = 0
l = 10
Q = 2.0
d = 1.5
g = 9.81
Omega = 0.65
initial_conditions = [theta_0, omega_0]

# Define the time points
t = np.linspace(0, 20, 200)

# Solve the ODE using `odeint`
solution = odeint(pendulum, initial_conditions, t, args=(Q, d, g, l, Omega))
theta = solution[:, 0]
omega = solution[:, 1]

# Finally, create two plots to visualize the results
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(t, theta)
plt.title("Angle (θ) as a Function of Time")
plt.xlabel("Time")
plt.ylabel("Angle (θ)")

plt.subplot(2, 1, 2)
plt.plot(t, omega)
plt.title("Angular Velocity (ω) as a Function of Time")
plt.xlabel("Time")
plt.ylabel("Angular Velocity (ω)")

plt.tight_layout()
plt.show()

3\. **FFT of a simple dataset**

Perform a periodicity analysis on the lynxs-hares population, i.e. determine what is the period of the population of these animals.

The dataset is the one downloaded at the beginning of Lecture 06:

 - `!wget https://www.dropbox.com/s/ebe1cnyd2gm836a/populations.txt -P data/`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftshift, fftfreq

# Load the dataset
file_path = "data/populations.txt"
data = np.loadtxt(file_path)

# Extract time and population data
time = data[:, 0]
populations = data[:, 1:]

# Perform FFT on the population
hares_fft = fft(populations[:, 0])
lynxes_fft = fft(populations[:, 1])

# Compute the frequency axis
hares_freq = fftfreq(len(hares_fft),1)
lynxes_freq = fftfreq(len(lynxes_fft),1)

# Plotting
fig = plt.figure()
plt.plot(fftshift(hares_freq), abs(fftshift(hares_fft)), 'b', label="Hares")
plt.plot(fftshift(lynxes_freq), abs(fftshift(lynxes_fft)),'y', label="Lynxes")
plt.title('Hares and lynxes FFT')
plt.xlabel('Frequence')
plt.ylabel('FFT')
plt.legend(loc="best")
plt.show()

# Select only positive frequencies
mask_hares = np.where(hares_freq>0)
mask_lynxes = np.where(hares_freq>0)
hfreq = hares_freq[mask_hares]
lfreq = lynxes_freq[mask_lynxes]
pos_l = np.abs(lynxes_fft)[mask_lynxes]
pos_h = np.abs(hares_fft)[mask_hares]

# Find the max frequency, then the period of the population
max_freq_h = hfreq[pos_h.argmax()]
max_freq_l = lfreq[pos_l.argmax()]
period_h = 1/max_freq_h
period_l = 1/max_freq_l
print("The period for the hares population is", period_h, "years.")
print("The period for the lynxes population is", period_l, "years.")

4\. **FFT of an image**

Write a filter that removes the periodic noise from the `moonlanding.png` image by using a 2-dimensional FFT.

* Import the image as a 2D numpy array using `plt.imread("images/moonlanding.png")`. Examine the image with `plt.imshow()`, which is heavily contaminated with periodic noise.
* Check the documentation of the `scipy.fftpack` package, and find the method that performs a 2D FFT. Plot the spectrum (Fourier transform of) the image. **Hint**: use `LogNorm` to plot the colors in log scale:
```Python
from matplotlib.colors import LogNorm
plt.imshow(image, norm=LogNorm(vmin=5))
```
* Inspect the spectrum, and try to locate the 2D regions of the power spectrum that contain the signal and those which contain the periodic noise. Use array slicing to set the noise regions to zero.
* Apply the inverse Fourier transform to plot the resulting image.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.fftpack import fft2, ifft2

# Import the image as a 2D numpy array
image = plt.imread("images/moonlanding.png")

# Examine the original image
plt.imshow(image, cmap='gray')
plt.title("Original image")
plt.show()

# Perform a 2D FFT on the image
image_fft = fft2(image)

# Plot the spectrum (Fourier transform of the image)
plt.imshow(np.abs(image_fft), norm=LogNorm(vmin=5), cmap='gray')
plt.title("Image spectrum")
plt.colorbar()
plt.show()

# Inspect the spectrum to identify the regions containing the signal and periodic noise
square = int(image.shape[0]/10)
image_fft[square:image.shape[0]-square, :] = 0
image_fft[:, square:image.shape[1]-square] = 0
plt.figure()
plt.imshow(abs(image_fft), norm=LogNorm(vmin=5), cmap='gray')
plt.colorbar()
plt.title("Filtered spectrum")
plt.show()

# Apply the inverse Fourier transform to obtain the filtered image
filtered_image = np.real(ifft2(image_fft))

# Plot the filtered image
plt.imshow(filtered_image, cmap='gray')
plt.title("Reconstructed image")
plt.show()